In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from ipyleaflet import (Map, GeoJSON, Marker, Popup)
from ipywidgets import HTML
import geopandas as gpd
from shapely.geometry import Point
import os
import json
import random
import requests




/Users/david/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# this df is for csv file of household income. 

df = pd.read_csv('../data/census-data/acs2021.csv')
df.head()

,geoid,name,B19013001,"B19013001, Error"
0,86000US10001,10001,101409.0,11816.0
1,86000US10002,10002,37093.0,3148.0
2,86000US10003,10003,137533.0,10077.0
3,86000US10004,10004,216017.0,10041.0
4,86000US10005,10005,197188.0,17558.0


In [3]:
df.duplicated().sum()


0

In [4]:
df.isnull().sum()

geoid                0
name                 0
B19013001           30
B19013001, Error    32
dtype: int64

In [5]:
## Drop null values + sanity check
df = df.dropna()
df.isnull().sum()

geoid               0
name                0
B19013001           0
B19013001, Error    0
dtype: int64

In [12]:
# Create map with geoJSON from censusReporter

with open('/Users/david/Documents/CunyTechPrep/mta-project/data-dashboard/data/census-data/acs2021_5yr_B19013_86000US11357.geojson', 'r') as f:
    data = json.load(f)

gdf = gpd.read_file('/Users/david/Documents/CunyTechPrep/mta-project/data-dashboard/data/census-data/acs2021_5yr_B19013_86000US11357.geojson')

# def random_color(feature):
#     return {
#         'color': 'black',
#         'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
#     }

# def income_color(feature):
#     # Access income data from the feature properties (assuming the income data is stored in a property named 'income')
#     income = feature['properties']['income']

#     # Define color ranges based on income levels
#     if income < 30000:
#         return {'fillColor': 'red'}  # Low income
#     elif 30000 <= income < 50000:
#         return {'fillColor': 'yellow'}  # Moderate income
#     else:
#         return {'fillColor': 'green'}  # Higher income

# m = Map(center=([40.730610, -73.935242]), zoom=10)

# geo_json = GeoJSON(
#     data=data,
#     style={
#         'opacity':1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
#     },
#     hover_style={
#         'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
#     },
#     style_callback=income_color
# )
# m.add_layer(geo_json)

# # Allow user to select radius on map
# dc = DrawControl(circle={'shapeOptions':{'color':'#0000FF'}})
# m.add_control(dc)
# m

## ---------------------------------- version 2 with uncentered centroids (markers for zipcodes)

# center = (40.7128, -74.0060)
# m = Map(center=center, zoom=10)

# Add GeoJSON layer
# geo_json = GeoJSON(data=data)
# m.add_layer(geo_json)

# Iterate through GeoJSON features to create markers

# for feature in data['features']:
#     # Extract zip code and income data
#     zipcode = feature['properties']['geoid']
#     income = feature['properties']['B19013001']

#     # Get the coordinates for the marker
#     coordinates = feature['geometry']['coordinates'][0][0][0]  # Change this based on your data structure

#     # Create marker and popup
#     marker = Marker(location=(coordinates[1], coordinates[0]))  # Latitude, Longitude
#     popup = Popup(
#         location=(coordinates[1], coordinates[0]),
#         child=HTML(value=f"<b>Zip Code:</b> {zipcode}<br><b>Income:</b> ${income}")
#     )
#     marker.popup = popup
#     m.add_layer(marker)

# m

## ------------------  version 3 below with centroids for markers


# gdf['centroid'] = gdf['geometry'].centroid

# # Create a map
# m = Map(center=(40.7128, -74.0060), zoom=10)

# # Iterate through GeoJSON features to create markers at centroids
# for index, row in gdf.iterrows():
#     zipcode = row['properties']['geoid']  # Replace 'ZIP_CODE' with the column name of zip codes in your GeoDataFrame
#     income = row['properties']['B19013001']  # Replace 'INCOME' with the column name of income in your GeoDataFrame

#     # Get centroid coordinates
#     centroid = row['centroid']
#     lon, lat = centroid.x, centroid.y

#     # Create marker and popup
#     marker = Marker(location=(lat, lon))
#     popup = Popup(
#         location=(lat, lon),
#         child=HTML(value=f"<b>Zip Code:</b> {zipcode}<br><b>Income:</b> ${income}")
#     )
#     marker.popup = popup
#     m.add_layer(marker)

# m


## Version 4 ------------

# Read the GeoJSON file
gdf = gpd.read_file('your_geojson_file.geojson')

# Calculate average income per zip code
average_income = gdf.groupby('name')['B19013001'].mean().reset_index()

# Calculate centroids for each zip code
gdf['centroid'] = gdf['geometry'].centroid

# Create a map
m = Map(center=(40.7128, -74.0060), zoom=10)

# Plot markers with popups
for idx, row in gdf.iterrows():
    # Get the centroid coordinates
    lon, lat = row['centroid'].x, row['centroid'].y
    
    # Find the average income for the current zip code
    income = average_income[average_income['name'] == row['name']]['B19013001'].values[0]
    
    # Create marker and popup
    marker = Marker(location=(lat, lon))
    popup = Popup(location=(lat, lon), child=f"Avg. Income: ${income:.2f}")
    marker.popup = popup
    m.add_layer(marker)

# Display the map
m



DriverError: your_geojson_file.geojson: No such file or directory